
## Modified from CIS 530 Homework Option 2 - Spring 2021




### Installing the Huggingface🤗 transformer package

In [ ]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "0"
!pip install transformers
!pip3 install sentencepiece
!pip install madgrad

     |████████████████████████████████| 2.1MB 14.9MB/s 
     |████████████████████████████████| 3.3MB 52.7MB/s 
     |████████████████████████████████| 901kB 46.9MB/s 
     |████████████████████████████████| 1.2MB 12.7MB/s 


### Import the important packages that we need

In [ ]:
import torch 
import numpy as np
import pandas as pd

### Mount your google drive 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
def get_metrics(true_labels, pred_labels):
    f1 = f1_score(true_labels, pred_labels, average='macro')
    acc = accuracy_score(true_labels, pred_labels)
    rec = recall_score(true_labels, pred_labels, average='macro')
    prec = precision_score(true_labels, pred_labels, average='macro')
    return f1, acc, rec, prec

### Download the Review Datset

Note that with the default code, the files are not saved in your google drive, which means they will get deleted after the session close. You can either re-run this cell for each new colab session, or you can save it to the mounted drive at `/content/drive`

In [ ]:
import json
import gzip

def parse(path):
  true = True
  false = False
  g = gzip.open(path, 'r')
  for l in g:
    yield json.dumps(eval(l))

def strict(path):
  max_reviews = 1689188
  iter = 5000
  counter = 0 
  f_path = path + ".json.gz"
  true = True
  false = False
  f = open(path+".json", 'w')
  # f.write("[")
  writable = "["
  # writable.join

  for l in parse(f_path):
    writable = writable + l + ',\n'
    counter = counter + 1
    if (counter % iter) == 0:
      print(f"On line{counter}, {counter/max_reviews} % done")
  writable = writable[:-2]
  writable = writable + "]"
  f.write(writable)

def fast_parse(path):
  # path = path + ".json.gz"
  max_reviews = 1689188
  iter = 5000
  counter = 0 
  all_reviews = []
  g = open(path, 'r')
  for line in g:
    d = json.loads(line)
    all_reviews.append(d)
    if (counter % iter) == 0:
      print(f"On line{counter}, {counter/max_reviews} % done")
    counter = counter + 1
  return all_reviews

In [ ]:
train_set = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/preprocessed_small_electronics.csv')
dev_set = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/preprocessed_small_electronics_test.csv')
train_set = train_set.to_dict(orient='records')
dev_set = dev_set.to_dict(orient='records')

In [ ]:
print(train_set[0]['reviewText'])
print("Number of review in training set: {}".format(len(train_set)))
print("Here's how one of the example looks like: {}".format(json.dumps(train_set[100])))

the camera is good ,  but the grip/stand could be a little better .   when i try to clip on the monitor ,  it falls off . 
Number of review in training set: 24880
Here's how one of the example looks like: {"reviewText": "do not buy this !  !  it completely stopped working after 7 months ,  and went in and out of connectivity the whole 7 months that it did work", "summary": "DO NOT BUY", "overall": 1.0, "pos_neg": 0, "neutrality": 1, "num_words": 26, "title_plus_review": "DO NOT BUY do not buy this !  !  it completely stopped working after 7 months ,  and went in and out of connectivity the whole 7 months that it did work"}


In [ ]:
import random

training_reviews = []
id = 0;
for  review in train_set:
  if 'reviewText' not in review:
    continue
  # if review['pos_neg']  > 1: 
    # continue
  training_reviews.append({
            "review_id": id,
            "review_text": str(review['reviewText']).encode('utf-8'),
            "label": review['overall']
        })
  
  id = id + 1
print("Number of reviews for training: {}".format(len(training_reviews)))

dev_reviews = []
id = 0;
for review in dev_set:
  #no Review text
  if 'reviewText' not in review:
    continue
  # if review['pos_neg']  > 1: 
    # continue
  dev_reviews.append({
            "review_id": id,
            "review_text": str(review['reviewText']).encode('utf-8'),
            "label": review['overall']
        })
  
  id = id + 1
print("Number of reviews for dev: {}".format(len(dev_reviews)))

Number of reviews for training: 19747
Number of reviews for dev: 4007


### Load Pretrained BERT Model
 

You can search for the available models [here](https://huggingface.co/models?search=bert).

You can find more examples of different use cases for BERT in the transformer github repo README -- https://github.com/huggingface/transformers


In [ ]:
from transformers import InputExample
from transformers import (WEIGHTS_NAME, BertConfig,
                          BertForSequenceClassification, BertTokenizer)
from transformers import glue_convert_examples_to_features as convert_examples_to_features
from transformers.optimization import AdamW, get_linear_schedule_with_warmup
import tqdm

from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,  TensorDataset)


#Bert-mini
bert_model_type = 'google/bert_uncased_L-4_H-256_A-4'  
# bert_model_type = "albert-base-v2"
# bert_model_type = 'google/bert_uncased_L-12_H-768_A-12'


bert_model = BertForSequenceClassification.from_pretrained(bert_model_type, num_labels = 5)
config = BertConfig.from_pretrained(bert_model_type)
tokenizer = BertTokenizer.from_pretrained(bert_model_type, num_labels = 5)


def init_model(bert_model_type):

  global bert_model
  global config
  global tokenizer
  bert_model = BertForSequenceClassification.from_pretrained(bert_model_type, num_labels = 5, hidden_dropout_prob = HYPER_PARAMS['dropout_prob'])
  config = BertConfig.from_pretrained(bert_model_type)
  tokenizer = BertTokenizer.from_pretrained(bert_model_type, num_labels = 5)
  bert_model.to('cuda')

  # from transformers import AutoTokenizer, AutoModelForPreTraining, AutoConfig
  # import sentencepiece

  # tokenizer = AutoTokenizer.from_pretrained("google/bigbird-roberta-base")
  # config = AutoConfig.from_pretrained("google/bigbird-roberta-base")
  # model = AutoModelForPreTraining.from_pretrained("google/bigbird-roberta-base")

Some weights of the model checkpoint at google/bert_uncased_L-4_H-256_A-4 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

### Convert examples to BERT input features



In [ ]:
relevance_label_mapping = {
    1.0: 1,
    2.0: 2,
    3.0: 3,
    4.0: 4,
    5.0: 5
}

def convert_sentence_pair_to_tensor_input(sentence_reviews, label_mapping):

    # STEP 1: convert each sentence 
    input_examples = []
    for review in sentence_reviews:
        current_label = review["label"] if "label" in review else print("dang")
        input_examples.append(
            InputExample(guid=review["review_id"], 
                         text_a=str(review["review_text"]), 
                         label=label_mapping[current_label])
        )

    label_list = [val for _, val in label_mapping.items()]
    features = convert_examples_to_features(input_examples,
                                                   tokenizer,
                                                   label_list=label_list,
                                                   max_length = 128,  
                                                   output_mode="classification")
    
    input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    labels = torch.tensor([f.label for f in features], dtype=torch.long)
    dataset = TensorDataset(input_ids, attention_mask, token_type_ids, labels)

    return dataset

In [ ]:
train_dataset = convert_sentence_pair_to_tensor_input(training_reviews, relevance_label_mapping)

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


In [ ]:
dev_dataset = convert_sentence_pair_to_tensor_input(dev_reviews, relevance_label_mapping)

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


### Choose your hyperparameters + model output directory


In [ ]:
HYPER_PARAMS = {
    "num_training_epoch": 3,
    "learning_rate": .001,        
    "training_batch_size": 32,    
    "eval_batch_size": 8,
    "max_grad_norm": 1.0,
    "num_warmup_steps": 0.1,
    "dropout_prob": 0.1
}

model_output_dir = "/content/drive/" 

### Fine-tune BERT model

Remember NOT to re-run this cell multiple times, without re-initializing the BERT model. Multiple runs will effectively train your model with more epochs than you intended!

In [ ]:
import tqdm

def train_model(optimizer):
  global bert_model
  global tokenizer
  global config
  torch.cuda.manual_seed(42)
  torch.manual_seed(42)
  random.seed(42)
  np.random.seed(42)
  bert_model.to("cuda")

  


  train_sampler = RandomSampler(train_dataset)
  train_dataloader = DataLoader(train_dataset, 
                                sampler=train_sampler, 
                                batch_size=HYPER_PARAMS["training_batch_size"])

  # optimizer = AdamW(bert_model.parameters(), 
  #                   lr=HYPER_PARAMS['learning_rate'], 
  #                   correct_bias=False)

  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps=HYPER_PARAMS['num_warmup_steps'], 
                                              num_training_steps=len(train_dataloader))


  global_step = 0
  tr_loss = 0.0
  bert_model.zero_grad()
  bert_model.train()

  for epc in range(HYPER_PARAMS["num_training_epoch"]):
      print("Epoch #{}: \n".format(epc))
      epoch_iterator = tqdm.notebook.tqdm(train_dataloader, desc="Training Steps")
      avg_loss_over_epoch = []
      for step, batch in enumerate(epoch_iterator):
          batch = tuple(t.to('cuda') for t in batch)
          inputs = {'input_ids': batch[0],
                    'attention_mask': batch[1],
                    'token_type_ids': batch[2],
                    'labels': batch[3]}

          outputs = bert_model(**inputs)
          loss = outputs[0]  
          loss.backward()
          torch.nn.utils.clip_grad_norm_(bert_model.parameters(), HYPER_PARAMS["max_grad_norm"])
          tr_loss += loss.item()

          optimizer.step()
          scheduler.step()
          bert_model.zero_grad()

In [ ]:
# # init_model(bert_model_type)
# optimizer = AdamW(bert_model.parameters(), 
#                     lr=HYPER_PARAMS['learning_rate'], 
#                     correct_bias=False)
# train_model(optimizer)

AttributeError: ignored

In [ ]:
# del bert_model
# del tokenizer
# del config
# init_model(bert_model_type)
# optimizer = AdamW(bert_model.parameters(), 
#                     lr=HYPER_PARAMS['learning_rate'], 
#                     correct_bias=False)
# train_model(optimizer)
# df = pd.DataFrame()
# optimizer_name = ('ug')
# test_dev(df)

Some weights of the model checkpoint at google/bert_uncased_L-4_H-256_A-4 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

Epoch #0: 




Epoch #1: 




Epoch #2: 




The accuracy on dev set = 0.0498


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Accuracy,F1,Model,Optimizer,Precision,Recall
0,0.0498,0.023719,google/bert_uncased_L-4_H-256_A-4,ug,0.01245,0.25


In [ ]:
import os

# This is where we mounted your google drive. 
# You might need to re-mount it if your session was closed half way through
output_dir = "/content/drive/My Drive/cis519_project/Hyper_mini_bert_1e3/" 

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

bert_model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
config.save_pretrained(output_dir)

def save_model(output_dir):
  # This is where we mounted your google drive. 
  # You might need to re-mount it if your session was closed half way through
  output_dir = "/content/drive/My Drive/cis519_project/Hyper_mini_bert_1e3/" 

  if not os.path.exists(output_dir):
      os.makedirs(output_dir)

  bert_model.save_pretrained(output_dir)
  tokenizer.save_pretrained(output_dir)
  config.save_pretrained(output_dir)

In [ ]:
def load_model():
  # output_dir = "/content/drive/My Drive/cis519_project/bin_model_mini/" 
  bert_model = BertForSequenceClassification.from_pretrained(output_dir, num_labels = 5)
  tokenizer = BertTokenizer.from_pretrained(output_dir)

  # Don't forget to move your model to GPU/CUDA after loading back from disk!
  bert_model = bert_model.to("cuda")

In [ ]:
#  Not Random Sample
def test_dev(results):
  global bert_model
  global tokenizer
  global config
  bert_model
  dev_sampler = SequentialSampler(dev_dataset)
  dev_dataloader = DataLoader(dev_dataset, 
                              sampler=dev_sampler, 
                              batch_size=HYPER_PARAMS["eval_batch_size"])

  predictions = None
  out_label_ids = None

  for batch in tqdm.notebook.tqdm(dev_dataloader, desc="Evaluating on Dev set..."):
      bert_model.eval()
      batch = tuple(t.to("cuda") for t in batch)
      inputs = {'input_ids': batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2],
                'labels': batch[3]}

      with torch.no_grad():
          outputs = bert_model(**inputs)
          logits = outputs[1] # This is 1x2 tensor, containing scores for both labels 
      if predictions is None:
          predictions = logits.detach().cpu().numpy()
          out_label_ids = inputs['labels'].detach().cpu().numpy()
      else:
          predictions = np.append(predictions, logits.detach().cpu().numpy(), axis=0)
          out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

  # whichever label gets higher score, we will predict that label
  predictions = np.argmax(predictions, axis=1)
  # print(predictions)
  # print(out_label_ids)


  f1, acc, rec, prec = get_metrics(out_label_ids, predictions)
  row = {'Model':bert_model_type, 'Optimizer':optimizer_name, 'LR': HYPER_PARAMS['learning_rate'], 'Dropout':HYPER_PARAMS['dropout_prob'], 'F1': f1, 'Accuracy': acc, 'Recall': rec, 'Precision':prec, }
  print("The accuracy on dev set = {}".format(acc))
  results = results.append(row, ignore_index = True)
  return results

0.5


In [ ]:
results = pd.DataFrame(columns=["Model", 'Optimizer', 'LR', 'Dropout','F1', 'Accuracy', 'Recall', 'Precision' ])

In [ ]:
from madgrad import MADGRAD
learning_rates = [1e-5, 3e-5, 5e-5, 1e-4, 1e-3, 5e-3]
dropouts_hidden = [0.1, 0.25, 0.5, 0.75, 1]
# adam = AdamW(bert_model.parameters(), 
#                     lr=HYPER_PARAMS['learning_rate'], 
#                     correct_bias=False)
# madgrad = madgrad(bert_model.parameters, lr=HYPER_PARAMS['learning_rate'])
bert_model_type='google/bert_uncased_L-4_H-256_A-4'
optimizers= ["AdamW", "Madgrad"]
for optimizer_name in optimizers:
  for rate in learning_rates:
    for drop in dropouts_hidden:
      HYPER_PARAMS["learning_rate"] = rate
      HYPER_PARAMS["dropout_prob"] = drop
      init_model(bert_model_type)
      if optimizer_name == "AdamW":
        optimizer = AdamW(bert_model.parameters(), 
                      lr=HYPER_PARAMS['learning_rate'], 
                      correct_bias=False)
      else:
        optimizer = MADGRAD(bert_model.parameters(), lr=HYPER_PARAMS['learning_rate'])
      train_model(optimizer)
      results = test_dev(results)
      print(results.head())
results.to_csv("/content/drive/MyDrive/cis519_final/Hyperparameters/elec_5_class_bert_mini_hyper.csv")
    

Some weights of the model checkpoint at google/bert_uncased_L-4_H-256_A-4 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

Epoch #0: 




Epoch #1: 




Epoch #2: 




The accuracy on dev set = 0.4470138749245928
                               Model Optimizer  ...    Recall  Precision
0  google/bert_uncased_L-4_H-256_A-4     AdamW  ...  0.430856   0.415311

[1 rows x 8 columns]


...

Epoch #0: 




Epoch #1: 




Epoch #2: 




The accuracy on dev set = 0.214558616529258
                               Model Optimizer  ...    Recall  Precision
0  google/bert_uncased_L-4_H-256_A-4     AdamW  ...  0.430856   0.415311
1  google/bert_uncased_L-4_H-256_A-4     AdamW  ...  0.417449   0.402231
2  google/bert_uncased_L-4_H-256_A-4     AdamW  ...  0.278314   0.290716
3  google/bert_uncased_L-4_H-256_A-4     AdamW  ...  0.208430   0.241619
4  google/bert_uncased_L-4_H-256_A-4     AdamW  ...  0.183589   0.110378

[5 rows x 8 columns]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
results

,Model,Optimizer,LR,Dropout,F1,Accuracy,Recall,Precision
0,google/bert_uncased_L-4_H-256_A-4,AdamW,0.00001,0.10,0.425127,0.6116,0.516514,0.402194
1,google/bert_uncased_L-4_H-256_A-4,AdamW,0.00001,0.25,0.409789,0.6004,0.497412,0.388978
2,google/bert_uncased_L-4_H-256_A-4,AdamW,0.00001,0.50,0.242455,0.4190,0.350451,0.296728
3,google/bert_uncased_L-4_H-256_A-4,AdamW,0.00001,0.75,0.103271,0.0942,0.247534,0.269782
4,google/bert_uncased_L-4_H-256_A-4,AdamW,0.00001,1.00,0.184538,0.6508,0.213600,0.186369
5,google/bert_uncased_L-4_H-256_A-4,AdamW,0.00003,0.10,0.472968,0.6248,0.562152,0.444648
6,google/bert_uncased_L-4_H-256_A-4,AdamW,0.00003,0.25,0.459723,0.6246,0.536089,0.431601
7,google/bert_uncased_L-4_H-256_A-4,AdamW,0.00003,0.50,0.391688,0.6396,0.455089,0.381390
8,google/bert_uncased_L-4_H-256_A-4,AdamW,0.00003,0.75,0.024940,0.0502,0.249631,0.345763
9,google/bert_uncased_L-4_H-256_A-4,AdamW,0.00003,1.00,0.185381,0.6562,0.215252,0.188175


In [ ]:
from madgrad import MADGRAD
learning_rates = [1e-5, 5e-5, 1e-4, 1e-3, 5e-3]
dropouts_hidden = [0.1, 0.25, 0.5, 0.75, 1]
bert_model_type = "bert-base-uncased"
optimizers= ["AdamW", "Madgrad"]
for optimizer_name in optimizers:
  for rate in learning_rates:
    for drop in dropouts_hidden:
      HYPER_PARAMS["learning_rate"] = rate
      HYPER_PARAMS["dropout_prob"] = drop
      init_model(bert_model_type)
      if optimizer_name == "AdamW":
        optimizer = AdamW(bert_model.parameters(), 
                      lr=HYPER_PARAMS['learning_rate'], 
                      correct_bias=False)
      else:
        optimizer = MADGRAD(bert_model.parameters(), lr=HYPER_PARAMS['learning_rate'])
      train_model(optimizer)
      results = test_dev(results)
results.to_csv("/content/drive/MyDrive/519_porject/hyper_results")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at


Epoch #0: 




Epoch #1: 




Epoch #2: 



KeyboardInterrupt: ignored

In [ ]:


HYPER_PARAMS = {
    "num_training_epoch": 3,
    "learning_rate": 1e-4,        # Suggested values -- [1e-5, 3e-5, 5e-5]
    "training_batch_size": 32,    # Suggested values -- [16, 32]
    "eval_batch_size": 8,
    "max_grad_norm": 1.0,
    "num_warmup_steps": 0.1
}
bert_model = BertForSequenceClassification.from_pretrained(bert_model_type, num_labels = 2)
config = BertConfig.from_pretrained(bert_model_type)
tokenizer = BertTokenizer.from_pretrained(bert_model_type, num_labels = 2)

import tqdm

bert_model.to('cuda')

train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, 
                              sampler=train_sampler, 
                              batch_size=HYPER_PARAMS["training_batch_size"])

optimizer = AdamW(bert_model.parameters(), 
                  lr=HYPER_PARAMS['learning_rate'], 
                  correct_bias=False)

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=HYPER_PARAMS['num_warmup_steps'], 
                                            num_training_steps=len(train_dataloader))


global_step = 0
tr_loss = 0.0
bert_model.zero_grad()
bert_model.train()

for epc in range(HYPER_PARAMS["num_training_epoch"]):
    print("Epoch #{}: \n".format(epc))
    epoch_iterator = tqdm.notebook.tqdm(train_dataloader, desc="Training Steps")
    avg_loss_over_epoch = []
    for step, batch in enumerate(epoch_iterator):
        batch = tuple(t.to('cuda') for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2],
                  'labels': batch[3]}

        outputs = bert_model(**inputs)
        loss = outputs[0]  # model outputs are always tuple in transformers (see doc)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(bert_model.parameters(), HYPER_PARAMS["max_grad_norm"])
        tr_loss += loss.item()

        optimizer.step()
        scheduler.step()
        bert_model.zero_grad()

import os

# This is where we mounted your google drive. 
# You might need to re-mount it if your session was closed half way through
output_dir = "/content/drive/My Drive/cis519_project/bin_model_mini_1e4/" 

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

bert_model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
config.save_pretrained(output_dir)

In [ ]:
!pip install madgrad

In [ ]:
from madgrad import madgrad
HYPER_PARAMS = {
    "num_training_epoch": 3,
    "learning_rate": 1e-3,        # Suggested values -- [1e-5, 3e-5, 5e-5]
    "training_batch_size": 32,    # Suggested values -- [16, 32]
    "eval_batch_size": 8,
    "max_grad_norm": 1.0,
    "num_warmup_steps": 0.1
}

bert_model = BertForSequenceClassification.from_pretrained(bert_model_type, num_labels = 2)
config = BertConfig.from_pretrained(bert_model_type)
tokenizer = BertTokenizer.from_pretrained(bert_model_type, num_labels = 2)

import tqdm

bert_model.to('cuda')


train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, 
                              sampler=train_sampler, 
                              batch_size=HYPER_PARAMS["training_batch_size"])

optimizer = madgrad(bert_model.parameters(), 
                  lr=HYPER_PARAMS['learning_rate'])

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=HYPER_PARAMS['num_warmup_steps'], 
                                            num_training_steps=len(train_dataloader))


global_step = 0
tr_loss = 0.0
bert_model.zero_grad()
bert_model.train()

for epc in range(HYPER_PARAMS["num_training_epoch"]):
    print("Epoch #{}: \n".format(epc))
    epoch_iterator = tqdm.notebook.tqdm(train_dataloader, desc="Training Steps")
    avg_loss_over_epoch = []
    for step, batch in enumerate(epoch_iterator):
        batch = tuple(t.to('cuda') for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2],
                  'labels': batch[3]}

        outputs = bert_model(**inputs)
        loss = outputs[0]  # model outputs are always tuple in transformers (see doc)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(bert_model.parameters(), HYPER_PARAMS["max_grad_norm"])
        tr_loss += loss.item()

        optimizer.step()
        scheduler.step()
        bert_model.zero_grad()

import os

# This is where we mounted your google drive. 
# You might need to re-mount it if your session was closed half way through
output_dir = "/content/drive/My Drive/cis519_project/bin_model_mini_1e3_madgrad/" 

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

bert_model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
config.save_pretrained(output_dir)

In [ ]:
# def predict_on_test_set():
#     """
#     Return a list of 0/1 prediction for each test example, in sequential order.
#     Please use the same label mapping as we have so far.
#     1 = True (Relevant)
#     0 = False (Not relevant)
#     """

#     test_dataset = convert_sentence_pair_to_tensor_input(test_sentence_pairs, relevance_label_mapping)

#     # We are not random sampling anymore when evaluating... As we want to keep the order 
#     test_sampler = SequentialSampler(test_dataset)
#     test_dataloader = DataLoader(test_dataset, 
#                                 sampler=test_sampler, 
#                                 batch_size=HYPER_PARAMS["eval_batch_size"])

#     predictions = None
#     out_label_ids = None

#     for batch in tqdm.notebook.tqdm(test_dataloader, desc="Evaluating on Test set..."):
#         bert_model.eval()
#         batch = tuple(t.to("cuda") for t in batch)
#         inputs = {'input_ids': batch[0],
#                   'attention_mask': batch[1],
#                   'token_type_ids': batch[2],
#                   'labels': batch[3]}

#         with torch.no_grad():
#             outputs = bert_model(**inputs)
#             logits = outputs[1] # This is 1x2 tensor, containing scores for both labels 

#         if predictions is None:
#             predictions = logits.detach().cpu().numpy()
#             out_label_ids = inputs['labels'].detach().cpu().numpy()
#         else:
#             predictions = np.append(predictions, logits.detach().cpu().numpy(), axis=0)
#             out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

#     # whichever label gets higher score, we will predict that label
#     predictions = np.argmax(predictions, axis=1)
    
#     list_of_predictions = predictions
    
#     return list_of_predictions


# # Feel free to change the save location as you like,
# # but please keep the file name as "relevance_test_predictions.txt"
# # So that the autograder will know what file to look for...
# test_result_output_path = "/content/drive/My Drive/cis530_perspective_hw/relevance_test_predictions.txt"

# test_predictions = predict_on_test_set()

# with open(test_result_output_path, 'w') as fout:
#     for pred in test_predictions: 
#         fout.write("{}\n".format(int(pred)))